In [7]:
import os
os.environ["JAVA_HOME"] = r"C:\Program Files\JAVA\jdk-22"

In [8]:
from src.utils import *
from src.graph_model import *
from src.owl2vec import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

In [10]:
dataset_name = 'OWL2DL-1'
# dataset_name = 'family'

In [11]:
experiments = get_experimets(dataset_name)

In [ ]:
for experiment in experiments:
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']
    
    subsumption_results = []
    membership_results = []
    link_prediction_results = []
    
    for i in range(5):
        model = OWL2Vec(
            file_name=file_name,
            iteration = i, 
            dataset_name=dataset_name,
            kge_model='transe',
            emb_dim=256,
            margin=0.1,
            weight_decay=0.0,
            batch_size=4096*8,
            lr=0.0001,
            num_negs=4,
            test_batch_size=32,
            epochs=500,
            device=device,
            seed=42,
            initial_tolerance=5
        )
        
        model.train()
        
        print(f'{file_name}:')
        metrics_subsumption, metrics_membership, metrics_link_prediction = model.test()
        print()

        subsumption_results.append(metrics_subsumption)
        membership_results.append(metrics_membership)
        link_prediction_results.append(metrics_link_prediction)
        
    avg_metrics_subsumption = [np.mean(values) for values in zip(*subsumption_results)]
    avg_metrics_membership = [np.mean(values) for values in zip(*membership_results)]
    avg_metrics_link_prediction = [np.mean(values) for values in zip(*link_prediction_results)]
    
    save_results(avg_metrics_subsumption, avg_metrics_membership, avg_metrics_link_prediction, f'models/results/owl2vec/{file_name}.txt') 